# BSS Spectral Sparsifier Implementation

In [1]:
import networkx as nx #This library allows an easier manipulation of graphs
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as LA
import scipy as sp
import random as rd

## Functions for getting different kind of matrices associated to graphs

In [13]:
#receive an integer n
#return an undirected unweighted complete graph of n vertices
def get_complete_graph(n):
    Kn = nx.complete_graph(n)
    return Kn

#receive a graph G
#return the adjacency matrix of G as numpy array object
def get_adjacency_matrix(G):
    A = nx.adjacency_matrix(G)
    return A.toarray()

#receive a graph G
#return the Laplacian matrix of G as numpy array object
def get_laplacian_matrix(G):
    L = nx.laplacian_matrix(G)
    return L.toarray()

#receive a graph G
#return the Signed Incidence matrix of G as numpy array object
def get_signed_incidence_matrix(G):
    B = nx.incidence_matrix(G, oriented=True)
    return B.toarray()

def get_weighted_diagonal_matrix(G):
    L = get_laplacian_matrix(G)
    n = len(G.nodes)
    W = [] 
    for i in range(n):
        for j in range(i+1,n):
            W.append(np.abs(L[i,j]))
    return np.diag(W)

#receive a graph G
#returns the vectors corresponding to every edge of G (the column vectos of the signed incidence matrix)
def get_rank_one_laplacian_decomposition(G):
    B = nx.incidence_matrix(G, oriented=True)
    m = G.number_of_edges()
    Vs = [] #vectors vi
    for i in range(m):
        u = B[:,i]
        Vs.append(u.toarray())
    return Vs

## Specific functions of the BSS algorithm

In [79]:
def upper_potential(A, u):
    I = np.identity(n)
    return np.trace(LA.inv((u*I)-A))

def lower_potential(A, l):
    I = np.identity(n)
    return np.trace(LA.inv(A-(l*I)))

def get_pseudoinverse(M):
    return sp.linalg.pinvh(M)

def get_matrix_square_root(M):
    return sp.linalg.sqrtm(M)

#Returns matrix V from proof of Theorem 1.1 from paper [2]
def get_matrix_V(B, Lpin, W=None):
    if W == None: return np.matmul(get_matrix_square_root(Lpin), B)#in case the graph is unweighted
    else: return np.matmul(get_matrix_square_root(Lpin), np.matmul(B, get_square_root(W)))
    
def upper_barrier_shift(A, v, u, deltau):
    I = np.identity(n)
    pot_u = upper_potential(A, u)
    pot_deltau = upper_potential(A, u+deltau)
    C = LA.matrix_power((u+deltau)*I-A, -2)
    D = LA.inv((u+deltau)*I-A)
    return np.dot(v, np.matmul(D, v)) + (np.dot(v, np.matmul(C, v))/pot_u-pot_deltau)

def lower_barrier_shift(A, v, l, deltal):
    I = np.identity(n)
    pot_l = lower_potential(A, l)
    pot_deltal = lower_potential(A, l+deltal)
    C = LA.matrix_power(A-(l+deltal)*I, -2)
    D = LA.inv(A-(l+deltal)*I)
    return (np.dot(v, np.matmul(C, v))/pot_deltal-pot_l) - np.dot(v, np.matmul(D, v))

## Here we create a graph and its associated matrices

In [108]:
n = 15
G = get_complete_graph(n)
L = get_laplacian_matrix(G)
B = get_signed_incidence_matrix(G)
Lpin = get_pseudoinverse(L)
V = get_matrix_V(B, Lpin)
print(V.shape)
m = len(G.edges)

(15, 105)


## Here we set the parameters of the BSS algorithm

In [109]:
#parameters
eps = 1/2
d = int(1/(eps**2))
delta_l = 1
delta_u = (np.sqrt(d)+1)/(np.sqrt(d)-1)
el = 1/np.sqrt(d)
eu = (np.sqrt(d)-1)/(d+np.sqrt(d))
li = -n/el
ui = n/eu
print(d)
print(ui)
print(li)

4
90.0
-30.0


## The BSS Spectral Sparsifier algorithm

In [110]:
def BSS(V, ui, li, delta_u, delta_l, d, n):
    S = np.zeros(m)
    A = np.zeros((n,n))
    q = 1
    while q <= d*n:
        i = q % m
        vi = V[:,i]
        Ui = upper_barrier_shift(A, vi, ui, delta_u)
        print(Ui)
        Li = lower_barrier_shift(A, vi, li, delta_l)
        print(Li)
        if Ui <= Li:
            t = (1/Ui)+(1/Li)
            S[i] += t
            ui += delta_u
            li += delta_l
            A += t*np.outer(vi,vi)
        q += 1
    return A, S

In [111]:
A, S = BSS(V, ui, li, delta_u, delta_l, d, n)

-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.15976413458203262
-0.5042911877394636
-0.1597641345820